<a href="https://colab.research.google.com/github/Clairekmkim/Clairekmkim/blob/main/Naver%20News%20Scraper%20with%20CSV%20Export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import requests
from bs4 import BeautifulSoup
import csv
from urllib.parse import quote_plus
import time
import os
import random

def scrape_naver_news(keyword1, keyword2, num_pages=20):
    news_data = []
    base_url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query={}&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start={}"

    query = quote_plus(f"{keyword1} {keyword2}")

    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0'
    ]

    session = requests.Session()

    for page in range(1, num_pages + 1):
        start = (page - 1) * 10 + 1
        url = base_url.format(query, start)

        headers = {
            'User-Agent': random.choice(user_agents),
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Referer': 'https://www.naver.com/',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
        }

        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = session.get(url, headers=headers, timeout=15)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, 'html.parser')

                news_items = soup.find_all('div', class_='news_area')

                if not news_items:
                    print(f"페이지 {page}에서 뉴스 항목을 찾을 수 없습니다.")
                    break

                for item in news_items:
                    title_elem = item.find('a', class_='news_tit')
                    content_elem = item.find('div', class_='news_dsc')

                    if title_elem and content_elem:
                        title = title_elem.text.strip()
                        content = content_elem.text.strip()
                        link = title_elem['href']

                        news_data.append({
                            'title': title,
                            'content': content,
                            'link': link
                        })

                print(f"페이지 {page} 완료 (항목 수: {len(news_items)})")
                break  # 성공적으로 데이터를 가져왔으므로 재시도 루프를 종료
            except requests.RequestException as e:
                print(f"페이지 {page} 스크래핑 중 오류 발생 (시도 {attempt + 1}/{max_retries}): {e}")
                if attempt == max_retries - 1:
                    print(f"페이지 {page} 스크래핑 실패")
                else:
                    time.sleep(random.uniform(5, 10))  # 재시도 전 더 긴 대기 시간

        # 페이지 간 랜덤한 지연 시간 (5-10초)
        time.sleep(random.uniform(5, 10))

    return news_data

def save_to_csv(data, filepath):
    if not data:
        print("저장할 데이터가 없습니다.")
        return

    os.makedirs(os.path.dirname(filepath), exist_ok=True)

    keys = data[0].keys()
    with open(filepath, 'w', newline='', encoding='utf-8-sig') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)

    print(f"{len(data)}개의 뉴스 항목이 '{filepath}' 파일에 저장되었습니다.")

if __name__ == "__main__":
    keyword1 = "리벨리온"
    keyword2 = "박성현"
    num_pages = 20  # 스크래핑할 페이지 수를 20으로 증가

    save_dir = input("결과를 저장할 디렉토리 경로를 입력하세요 (엔터를 누르면 현재 디렉토리에 저장됩니다): ").strip()
    if not save_dir:
        save_dir = os.getcwd()

    filename = f"naver_news_{keyword1}_{keyword2}.csv"
    filepath = os.path.join(save_dir, filename)

    news_data = scrape_naver_news(keyword1, keyword2, num_pages)

    if news_data:
        save_to_csv(news_data, filepath)
    else:
        print("스크래핑된 뉴스 데이터가 없습니다.")

    print(f"현재 작업 디렉토리: {os.getcwd()}")
    print("현재 디렉토리의 파일 목록:")
    print(os.listdir())

# Google Colab 사용 시
from google.colab import files
files.download(filepath)

결과를 저장할 디렉토리 경로를 입력하세요 (엔터를 누르면 현재 디렉토리에 저장됩니다): 
페이지 1 완료 (항목 수: 10)
페이지 2 완료 (항목 수: 10)
페이지 3 완료 (항목 수: 10)
페이지 4 완료 (항목 수: 10)
페이지 5 완료 (항목 수: 10)
페이지 6 완료 (항목 수: 10)
페이지 7 완료 (항목 수: 10)
페이지 8 완료 (항목 수: 10)
페이지 9 완료 (항목 수: 10)
페이지 10 완료 (항목 수: 10)
페이지 11 완료 (항목 수: 10)
페이지 12 완료 (항목 수: 10)
페이지 13 완료 (항목 수: 10)
페이지 14 완료 (항목 수: 10)
페이지 15 완료 (항목 수: 10)
페이지 16 완료 (항목 수: 10)
페이지 17 완료 (항목 수: 10)
페이지 18 완료 (항목 수: 10)
페이지 19 완료 (항목 수: 10)
페이지 20 완료 (항목 수: 10)
200개의 뉴스 항목이 '/content/naver_news_리벨리온_박성현.csv' 파일에 저장되었습니다.
현재 작업 디렉토리: /content
현재 디렉토리의 파일 목록:
['.config', 'naver_news_리벨리온_박성현.csv', 'sample_data']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [15]:
import requests
from bs4 import BeautifulSoup
import csv
from urllib.parse import quote_plus
import time
import os
import random

def scrape_naver_news(keyword1, keyword2, num_pages=20):
    news_data = []
    base_url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query={}&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start={}"

    query = quote_plus(f"{keyword1} {keyword2}")

    user_agents = [
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0 Safari/605.1.15',
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0'
    ]

    session = requests.Session()

    for page in range(1, num_pages + 1):
        start = (page - 1) * 10 + 1
        url = base_url.format(query, start)

        headers = {
            'User-Agent': random.choice(user_agents),
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'Referer': 'https://www.naver.com/',
            'DNT': '1',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
        }

        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = session.get(url, headers=headers, timeout=15)
                response.raise_for_status()
                soup = BeautifulSoup(response.text, 'html.parser')

                news_items = soup.find_all('div', class_='news_area')

                if not news_items:
                    print(f"페이지 {page}에서 뉴스 항목을 찾을 수 없습니다.")
                    break

                for item in news_items:
                    title_elem = item.find('a', class_='news_tit')
                    content_elem = item.find('div', class_='news_dsc')
                    date_elem = item.find('span', class_='info')  # 날짜 정보를 포함하는 요소

                    if title_elem and content_elem:
                        title = title_elem.text.strip()
                        content = content_elem.text.strip()
                        link = title_elem['href']
                        date = date_elem.text.strip() if date_elem else "날짜 정보 없음"

                        news_data.append({
                            'title': title,
                            'content': content,
                            'link': link,
                            'date': date  # 날짜 정보 추가
                        })

                print(f"페이지 {page} 완료 (항목 수: {len(news_items)})")
                break  # 성공적으로 데이터를 가져왔으므로 재시도 루프를 종료
            except requests.RequestException as e:
                print(f"페이지 {page} 스크래핑 중 오류 발생 (시도 {attempt + 1}/{max_retries}): {e}")
                if attempt == max_retries - 1:
                    print(f"페이지 {page} 스크래핑 실패")
                else:
                    time.sleep(random.uniform(5, 10))  # 재시도 전 더 긴 대기 시간

        # 페이지 간 랜덤한 지연 시간 (5-10초)
        time.sleep(random.uniform(5, 10))

    return news_data

def save_to_csv(data, filepath):
    if not data:
        print("저장할 데이터가 없습니다.")
        return

    os.makedirs(os.path.dirname(filepath), exist_ok=True)

    keys = data[0].keys()
    with open(filepath, 'w', newline='', encoding='utf-8-sig') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(data)

    print(f"{len(data)}개의 뉴스 항목이 '{filepath}' 파일에 저장되었습니다.")

if __name__ == "__main__":
    keyword1 = "리벨리온"
    keyword2 = "박성현"
    num_pages = 20

    save_dir = input("결과를 저장할 디렉토리 경로를 입력하세요 (엔터를 누르면 현재 디렉토리에 저장됩니다): ").strip()
    if not save_dir:
        save_dir = os.getcwd()

    filename = f"naver_news_{keyword1}_{keyword2}.csv"
    filepath = os.path.join(save_dir, filename)

    news_data = scrape_naver_news(keyword1, keyword2, num_pages)

    if news_data:
        save_to_csv(news_data, filepath)
    else:
        print("스크래핑된 뉴스 데이터가 없습니다.")

    print(f"현재 작업 디렉토리: {os.getcwd()}")
    print("현재 디렉토리의 파일 목록:")
    print(os.listdir())

# Google Colab 사용 시
from google.colab import files
files.download(filepath)

결과를 저장할 디렉토리 경로를 입력하세요 (엔터를 누르면 현재 디렉토리에 저장됩니다): 
페이지 1 완료 (항목 수: 10)
페이지 2 완료 (항목 수: 10)
페이지 3 완료 (항목 수: 10)
페이지 4 완료 (항목 수: 10)
페이지 5 완료 (항목 수: 10)
페이지 6 완료 (항목 수: 10)
페이지 7 완료 (항목 수: 10)
페이지 8 완료 (항목 수: 10)
페이지 9 완료 (항목 수: 10)
페이지 10 완료 (항목 수: 10)
페이지 11 완료 (항목 수: 10)
페이지 12 완료 (항목 수: 10)
페이지 13 완료 (항목 수: 10)
페이지 14 완료 (항목 수: 10)
페이지 15 완료 (항목 수: 10)
페이지 16 완료 (항목 수: 10)
페이지 17 완료 (항목 수: 10)
페이지 18 완료 (항목 수: 10)
페이지 19 완료 (항목 수: 10)
페이지 20 완료 (항목 수: 10)
200개의 뉴스 항목이 '/content/naver_news_리벨리온_박성현.csv' 파일에 저장되었습니다.
현재 작업 디렉토리: /content
현재 디렉토리의 파일 목록:
['.config', 'naver_news_리벨리온_박성현.csv', 'sample_data']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>